In [1]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# MIMIC - Triage

This notebook runs the analysis on the MIMIC emergency data by leveraging experts' agreement
1. Explore a model build on data ignoring experts 
2. Compute agreement between experts using influence function
3. Retrain the model on the set of labels for which experts strongly agree

The current analysis uses multi layer perceptrons in a single train / test split.

### Data

Reopen the data created with the notebook in `data/`

In [2]:
triage = pd.read_csv('../data/triage_clean.csv', index_col = [0, 1])
covariates, target, nurses = triage.drop(columns = ['D', 'Y1', 'Y2', 'YC', 'acuity', 'nurse']), triage[['D', 'Y1', 'Y2', 'YC']], triage['nurse']

Split data in a 80% train, 20% test

In [3]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(covariates, target, nurses, test_size = 0.2, random_state = 42)

### Modelling

In [4]:
# Model's characteristics
params = {'layers': []} # If = [] equivalent to a simple logistic regression

# Amalgation parameters
rho = 0.05 # Control which point to consider from a confience point of view
pi_1 = 4.0 # Control criterion on centre mass metric
pi_2 = 0.8 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

##### 1. Train on decision

This model models the nurse decision based on covariates

In [5]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['D'], nur_train)

Loss: 0.646:  69%|██████▉   | 688/1000 [10:23<04:42,  1.10it/s]


In [6]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.3726568584276959

In [7]:
# Yc performance
roc_auc_score(tar_test['YC'], model.predict(cov_test))

0.47047131828851174

##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [8]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params)

Loss: 0.648:   6%|▋         | 64/1000 [00:41<10:00,  1.56it/s]


In [9]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

/home/vincent/Desktop/Cambridge/Project/Influence/example/../model/amalgamation.py:55: RuntimeWarning: invalid value encountered in double_scalars
  center = np.dot(inf_sorted, np.arange(len(influence_point))) / np.sum(inf_sorted)
/home/vincent/Desktop/Cambridge/Project/Influence/example/../model/amalgamation.py:63: RuntimeWarning: invalid value encountered in double_scalars
  opposing = np.max([inf_pos.sum(), - inf_neg.sum()]) / total


In [10]:
# Apply criteria on amalgamation
high_conf = (predictions > (1 - rho)) | (predictions < rho)
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [11]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1'].copy()
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

/tmp/ipykernel_13443/148226879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tar_train['Ya'] = tar_train['Y1'].copy()
/home/vincent/miniconda3/envs/Jupyter/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [12]:
index_amalg = tar_train['D'] | high_agr_correct

##### 3. Updated model

In [20]:
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg])

Loss: 0.623:   2%|▏         | 23/1000 [00:08<06:13,  2.62it/s]


In [21]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.6490108254809583

In [22]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

0.5658628634431628

##### 4. Train on observed data

In [16]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['Y1'], nur_train)

Loss: 0.687:  13%|█▎        | 128/1000 [01:55<13:04,  1.11it/s]


In [17]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

0.6532088012868359

In [18]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

0.5464182542438537